In [1]:
import pandas as pd
import numpy as np
import demandlib.bdew as bdew
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
df = pd.read_json(r"C:\Users\codingPurposes\Desktop\Scripts\raw_datasets\merged_lean_solar_data.json", orient="split")
df = df.drop(['lat', 'population', 'lon', 'solar_output_list', 'solar_data_list', ], axis=1)

In [ ]:
yearly_demand_db = pd.read_csv(r"C:\Users\codingPurposes\Desktop\Scripts\raw_datasets\heat_demand_DATABASE.csv")

# Fixing 's-Gravenhage's 1st character error
indexOfDenHaag = yearly_demand_db.loc[yearly_demand_db["Row Labels"] == "'s-Gravenhage"].index[0]
yearly_demand_db.at[indexOfDenHaag,"Row Labels"] = "s-Gravenhage"

yearly_demand_db = yearly_demand_db.rename(index=str, columns={"Row Labels": "gemeentenaam"})

In [ ]:
df_merge = pd.merge(df, yearly_demand_db, on='gemeentenaam')
df_merge.head()

In [ ]:
gdf_demand_df = pd.DataFrame(index=range(87600), columns=list(df_merge.iloc[:,0]))

In [ ]:
# generate data for 10 years:

for i in range(len(df_merge)):
    print(i)
    for y in range(10):
        demand = pd.DataFrame(index=pd.date_range(pd.datetime(2006+y, 1, 1, 0), periods=8760, freq='H'))

        lowerThreshold = (0 + 8760*y)
        upperThreshold = (8760*y+8760)
        muniName = list(df_merge.iloc[:,0])[i]
        
        temperature = pd.DataFrame(df_merge.iloc[i,1][lowerThreshold:upperThreshold])
        annualDemand = df_merge.iloc[i,4] * 1000

        demandValues = bdew.HeatBuilding(demand.index, holidays={}, temperature=temperature,
                                          shlp_type='ghd', wind_class=0, annual_heat_demand=annualDemand,
                                          name='ghd').get_bdew_profile()

        gdf_demand_df.loc[lowerThreshold:upperThreshold-1, muniName] = list(demandValues)

In [ ]:
gdf_demand_df.to_json(r"C:\Users\codingPurposes\Desktop\Scripts\processed_datasets\gdf_heat_demand.json", orient="split")

In [ ]:
ggb_demand_df = pd.DataFrame(index=range(87600), columns=list(df_merge.iloc[:,0]))
# generate data for 10 years:

for i in range(len(df_merge)):
    print(i)
    for y in range(10):
        demand = pd.DataFrame(index=pd.date_range(pd.datetime(2006+y, 1, 1, 0), periods=8760, freq='H'))

        lowerThreshold = (0 + 8760*y)
        upperThreshold = (8760*y+8760)
        muniName = list(df_merge.iloc[:,0])[i]
        
        temperature = pd.DataFrame(df_merge.iloc[i,1][lowerThreshold:upperThreshold])
        annualDemand = df_merge.iloc[i,3] * 1000

        demandValues = bdew.HeatBuilding(demand.index, holidays={}, temperature=temperature,
                                          shlp_type='ggb', wind_class=0, annual_heat_demand=annualDemand,
                                          name='ggb').get_bdew_profile()

        ggb_demand_df.loc[lowerThreshold:upperThreshold-1, muniName] = list(demandValues)

In [ ]:
ggb_demand_df.to_json(r"C:\Users\codingPurposes\Desktop\Scripts\processed_datasets\ggb_heat_demand.json", orient="split")

In [ ]:
hh_demand_df = pd.DataFrame(index=range(87600), columns=list(df_merge.iloc[:,0]))
# generate data for 10 years:

for i in range(len(df_merge)):
    print(i)
    for y in range(10):
        demand = pd.DataFrame(index=pd.date_range(pd.datetime(2006+y, 1, 1, 0), periods=8760, freq='H'))

        lowerThreshold = (0 + 8760*y)
        upperThreshold = (8760*y+8760)
        muniName = list(df_merge.iloc[:,0])[i]
        
        temperature = pd.DataFrame(df_merge.iloc[i,1][lowerThreshold:upperThreshold])
        annualDemand = df_merge.iloc[i,2] * 1000

        demandValues = bdew.HeatBuilding(demand.index, holidays={}, temperature=temperature,
                                          shlp_type='MFH', building_class=1, wind_class=0, 
                                          annual_heat_demand=annualDemand, name='MFH').get_bdew_profile()

        hh_demand_df.loc[lowerThreshold:upperThreshold-1, muniName] = list(demandValues)

In [ ]:
hh_demand_df.to_json(r"C:\Users\codingPurposes\Desktop\Scripts\processed_datasets\hh_heat_demand.json", orient="split")

In [ ]:
total_heat_demand_df = gdf_demand_df.add(ggb_demand_df, fill_value=0)
total_heat_demand_df = total_heat_demand_df.add(hh_demand_df, fill_value=0)

### Checkpoint 1

In [2]:
hh_demand_df = pd.read_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\hh_heat_demand.json", orient="split")
ggb_demand_df = pd.read_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\ggb_heat_demand.json", orient="split")
gdf_demand_df = pd.read_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\gdf_heat_demand.json", orient="split")

total_heat_demand_df = gdf_demand_df.add(ggb_demand_df, fill_value=0)
total_heat_demand_df = total_heat_demand_df.add(hh_demand_df, fill_value=0)

In [3]:
# denemeee = pd.DataFrame(columns=["the Hauge"])
# denemeee["the Hauge"] = pd.Series(total_heat_demand_df.iloc[:8760,0])/1000
# ax = denemeee.plot(figsize=[16,6])
# ax.set_xlabel("Hours", fontsize= 14)
# ax.set_ylabel("Heat demand (MWh)", fontsize= 14)
# plt.title("Heat demand of 's-Gravenhage for the year 2020", {'fontsize': 18})
# fig = ax.get_figure()
# fig.savefig(r'C:\Users\codingPurposes\Desktop\for_report\denhaagHeatDemand2020.png')


In [4]:
# denemeee = pd.DataFrame(columns=["the Hauge"])
# denemeee["the Hauge"] = pd.Series(total_heat_demand_df.iloc[:24,0])/1000
# ax = denemeee.plot(figsize=[16,6])
# ax.set_xlabel("Hours", fontsize= 14)
# ax.set_ylabel("Heat demand (MWh)", fontsize= 14)
# plt.title("Daily profile of 's-Gravenhage's heat demand", {'fontsize': 18})

# fig = ax.get_figure()
# fig.savefig(r'C:\Users\codingPurposes\Desktop\for_report\denhaagHeatDemandDaily.png')


In [5]:
# f, ax = plt.subplots(figsize=(16, 8))
# ydata = electricity_demand["NL_load"]
# linedata = df["Load_pred"]
# plt.xlabel("Hours", {'fontsize': 14})
# plt.ylabel("Electricity demand (MWh)", {'fontsize': 14})
# plt.title("Collected electricity demand data (2006-2017)", {'fontsize': 18})
# ax.plot(ydata)
# ax.plot(linedata)
# ax.set_ylim(ymin=0)
# plt.savefig(r'C:\Users\codingPurposes\Desktop\for_report\Collected electricity demand data (2006-2017).png')
# plt.show(f)

In [6]:
total_heat_demand_df = pd.concat([total_heat_demand_df, total_heat_demand_df, total_heat_demand_df], ignore_index=True)

In [7]:
for d in tqdm(range(3650*3)):
    for h in range(24):
        total_heat_demand_df.loc[(d*24)+h, "dayIndex"] = d

representativeDays = [149, 336, 1255, 1381, 1414, 1486, 2239, 3284, 3462]
listHours = []

for y in range(3):
    for d in representativeDays:
        for h in range(24):
            listHours.append(24*(d+y*3650) + h)

total_heat_demand_df = total_heat_demand_df[total_heat_demand_df.index.isin(listHours)]
total_heat_demand_df = total_heat_demand_df.reset_index(drop=True)

100%|██████████| 10950/10950 [00:48<00:00, 226.58it/s]


In [8]:
three_hourly_heat_demand = pd.DataFrame(columns=list(total_heat_demand_df))

for muni in tqdm(list(three_hourly_heat_demand)):
    if muni != "dayIndex":
        emptyListValue = []
        
        for i in range(int(len(total_heat_demand_df)/3)):
            value = total_heat_demand_df.loc[1+(3*i), muni] * 3
            emptyListValue.append(value)
            
        three_hourly_heat_demand[muni] = emptyListValue
    
    else:
        emptyListDay = []
        for i in range(int(len(total_heat_demand_df)/3)):
            day = total_heat_demand_df.loc[1+(3*i), muni]
            emptyListDay.append(day)

        three_hourly_heat_demand[muni] = emptyListDay
        
daily_heat_demand = pd.DataFrame(columns=list(three_hourly_heat_demand))

for muni in tqdm(list(daily_heat_demand)):
    if muni != "dayIndex":
        for i in range(int(len(three_hourly_heat_demand)/8)):
            daily_heat_demand.loc[i, muni] = list(three_hourly_heat_demand[muni].loc[(0 + 8*i):(7 + 8*i)])
    else:
        for i in range(int(len(three_hourly_heat_demand)/8)):
            daily_heat_demand.loc[i, muni] = three_hourly_heat_demand[muni].loc[(0 + 8*i)]

100%|██████████| 356/356 [00:05<00:00, 65.12it/s]


In [ ]:
daily_heat_demand.to_json(r'C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\daily_3hourly_heatDemand.json',orient='split')

In [ ]:
# total_heat_demand_df.to_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\total_heat_demand_10years.json",orient="split")

In [ ]:
# total_heat_demand_df = pd.concat([total_heat_demand_df, total_heat_demand_df, total_heat_demand_df], ignore_index=True)

In [ ]:
# total_heat_demand_df.head()

In [ ]:
# three_hourly_heat_demand = pd.DataFrame(columns=list(total_heat_demand_df))

# for muni in list(three_hourly_heat_demand):
#     print(muni)
#     emptyList = []
    
#     for i in range(int(len(total_heat_demand_df)/3)):
#         value = total_heat_demand_df.loc[1+(3*i), muni]
#         emptyList.append(value * 3)
        
#     three_hourly_heat_demand[muni] = emptyList

In [ ]:
# three_hourly_heat_demand_2years = pd.DataFrame(columns=list(total_heat_demand_df))

# for muni in list(three_hourly_heat_demand):
#     print(muni)
#     emptyList = []
    
#     for i in range(15):
#         for t in range(2920):
#             value = three_hourly_heat_demand.loc[(i*2*2920)+t, muni]
#             emptyList.append(value)
                              
#     three_hourly_heat_demand_2years[muni] = emptyList

In [ ]:
# three_hourly_heat_demand_2years.to_json(r'C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\three_hourly_heat_demand_2years.json',orient='split')

In [ ]:
# daily_heat_demand = pd.DataFrame(columns=list(three_hourly_heat_demand_2years))

# for i in range(len(list(daily_heat_demand))):
#     relatedMunicipality = pd.DataFrame(data=three_hourly_heat_demand_2years.iloc[:,i])
#     relatedMunicipalityDaily = pd.DataFrame(columns=list(relatedMunicipality))
#     municipality = list(relatedMunicipality)[0]
#     print(municipality)
#     for i in range(int(len(relatedMunicipality)/8)):
#         relatedMunicipalityDaily.loc[i, municipality] = list(relatedMunicipality.loc[(0 + 8*i):(7 + 8*i), municipality])
        
#     daily_heat_demand[municipality]= relatedMunicipalityDaily[municipality]

In [ ]:
# # Exporting daily heat demand of municipalities:
# daily_heat_demand.to_json(r'C:\Users\codingPurposes\Desktop\processedData\processed_heat_demand\daily_3hourly_heatDemand_2years_identical.json',orient='split')